# 🚀 TESTGPT – 10-Minute AI-Testing Challenge
### Run **one cell** and watch agents auto-create tests from a YAML spec!

## 🔧 Step 0 – Install & Secrets
*(Double-click ➜ add your Groq key)*

In [ ]:
%%capture
!pip install openai-agents pyyaml

In [ ]:
import os, json, yaml, re
from openai import AsyncOpenAI
from agents import Agent, Runner, OpenAIChatCompletionsModel

# 🔑 Add your Groq key here
os.environ["GROQ_API_KEY"] = ""  # <-- paste your key inside the quotes

## 📝 Step 1 – One-Click Spec (YAML)

In [ ]:
%%writefile sample.yaml
product: SpaceRockets
version: "1.0"
openapi:
  openapi: 3.0.0
  info:
    title: Rocket Store API
    version: 1.0.0
  paths:
    /rockets/{id}:
      get:
        summary: Get one rocket
        parameters:
          - name: id
            in: path
            required: true
            schema: {type: integer}
        responses:
          "200": {description: Rocket found}
user_stories:
  - As a customer I can view rocket details
nfr:
  - p95 latency < 300 ms
  - support 1000 concurrent users
data_flow:
  source: sales_csv
  transform: clean + enrich
  target: analytics_warehouse

## 🤖 Step 2 – Agents in 30 Lines

In [ ]:
# Groq rocket
groq_model = OpenAIChatCompletionsModel(
    model="llama-3.1-8b-instant",
    openai_client=AsyncOpenAI(
        base_url="https://api.groq.com/openai/v1",
        api_key=os.getenv("GROQ_API_KEY")
    )
)

# Planning agent
planning_agent = Agent(
    name="PlanningAgent",
    instructions=(
        "Reply ONLY with a JSON list like: [{\"id\":\"API-S1\",\"risk\":0.3,\"agent\":\"api\"}]"
    ),
    model=groq_model
)

# Orchestrator agent
orchestrator = Agent(
    name="Orchestrator",
    instructions="Forward spec to PlanningAgent and save JSON.",
    model=groq_model
)

## 🏁 Step 3 – Press Run & Watch Magic

In [ ]:
spec = yaml.safe_load(open("sample.yaml"))
raw = yaml.dump(spec)

plan = Runner.run_sync(planning_agent, raw).final_output
print("📋 PlanningAgent output:")
print(plan)

# Clean JSON block
match = re.search(r"```json\s*(.*?)\s*```", plan, re.DOTALL)
json_str = match.group(1) if match else plan.strip()

os.makedirs("tests", exist_ok=True)
with open("tests/backlog.json", "w") as f:
    json.dump(json.loads(json_str), f, indent=2)

print("✅ Saved tests/backlog.json — ready for next robots!")

## 📣 Share It on LinkedIn
- ✅ Upload this notebook
- ✅ Screenshot the JSON output
- ✅ Tag `#TESTGPT #10MinChallenge #AIQA`